In [2]:
!pip install --upgrade git+https://github.com/m-wrzr/populartimes openmeteo-requests requests-cache retry-requests

  Cloning https://github.com/m-wrzr/populartimes to /tmp/pip-req-build-krlped0b
  Running command git clone --filter=blob:none --quiet https://github.com/m-wrzr/populartimes /tmp/pip-req-build-krlped0b
  Resolved https://github.com/m-wrzr/populartimes to commit 989f66b63dd1b6a144c5ad4a7c3a200f7f8649ee
  Preparing metadata (setup.py) ... done


In [1]:
from google.colab import userdata

GCP_API_KEY = userdata.get('GCP_API_KEY')

In [ ]:
import requests
import pandas as pd
import time
import populartimes

BASE_URL = "https://places.googleapis.com/v1/places:searchText"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": GCP_API_KEY,
    "X-Goog-FieldMask": (
        "places.id,"
        "places.displayName,"
        "places.formattedAddress,"
        "places.shortFormattedAddress,"
        "places.location,"
        "places.nationalPhoneNumber,"
        "places.internationalPhoneNumber,"
        "places.websiteUri,"
        "places.rating,"
        "places.userRatingCount,"
        "places.priceLevel,"
        "places.businessStatus,"
        "places.types,"
        "places.currentOpeningHours,"
        "places.regularOpeningHours,"
        "places.editorialSummary,"
        "places.accessibilityOptions,"
        "places.parkingOptions,"
        "places.paymentOptions,"
        "places.takeout,"
        "places.delivery,"
        "places.dineIn,"
        "places.googleMapsUri,"
        "nextPageToken"
    )
}

type_of_restaurans = [
    "Korean",
    "Mexican",
    "American",
    "German",
    "Italian",
    "Vietnamese",
    "Hungarian",
    "Indian",
    "Turkish"
]

districts = ['Mitte, 10115', 'Mitte, 10559', 'Mitte, 13355', 'Mitte, 10117', 'Mitte, 10623', 'Mitte, 13357', 'Mitte, 10119', 'Mitte, 10785', 'Mitte, 13359', 'Mitte, 10178', 'Mitte, 10787', 'Mitte, 13405', 'Mitte, 10179', 'Mitte, 10963', 'Mitte, 13407', 'Mitte, 10435', 'Mitte, 10969', 'Mitte, 13409', 'Mitte, 10551', 'Mitte, 13347', 'Mitte, 10553', 'Mitte, 13349', 'Mitte, 10555', 'Mitte, 13351', 'Mitte, 10557', 'Mitte, 13353', 'Friedrichshain-Kreuzberg, 10179', 'Friedrichshain-Kreuzberg, 10967', 'Friedrichshain-Kreuzberg, 10243', 'Friedrichshain-Kreuzberg, 10969', 'Friedrichshain-Kreuzberg, 10245', 'Friedrichshain-Kreuzberg, 10997', 'Friedrichshain-Kreuzberg, 10247', 'Friedrichshain-Kreuzberg, 10999', 'Friedrichshain-Kreuzberg, 10249', 'Friedrichshain-Kreuzberg, 12045', 'Friedrichshain-Kreuzberg, 10367', 'Friedrichshain-Kreuzberg, 10785', 'Friedrichshain-Kreuzberg, 10961', 'Friedrichshain-Kreuzberg, 10963', 'Friedrichshain-Kreuzberg, 10965', 'Pankow, 10119', 'Pankow, 13053', 'Pankow, 13187', 'Pankow, 10247', 'Pankow, 13086', 'Pankow, 13189', 'Pankow, 10249', 'Pankow, 13088', 'Pankow, 10405', 'Pankow, 13089', 'Pankow, 10407', 'Pankow, 13125', 'Pankow, 10409', 'Pankow, 13127', 'Pankow, 10435', 'Pankow, 13129', 'Pankow, 10437', 'Pankow, 13156', 'Pankow, 10439', 'Pankow, 13158', 'Pankow, 13051', 'Pankow, 13159', 'Charlottenburg-Wilmersdorf, 10553', 'Charlottenburg-Wilmersdorf, 10711', 'Charlottenburg-Wilmersdorf, 13353', 'Charlottenburg-Wilmersdorf, 14193', 'Charlottenburg-Wilmersdorf, 10585', 'Charlottenburg-Wilmersdorf, 10713', 'Charlottenburg-Wilmersdorf, 13597', 'Charlottenburg-Wilmersdorf, 14195', 'Charlottenburg-Wilmersdorf, 10587', 'Charlottenburg-Wilmersdorf, 10715', 'Charlottenburg-Wilmersdorf, 13627', 'Charlottenburg-Wilmersdorf, 14197', 'Charlottenburg-Wilmersdorf, 10589', 'Charlottenburg-Wilmersdorf, 10717', 'Charlottenburg-Wilmersdorf, 13629', 'Charlottenburg-Wilmersdorf, 14199', 'Charlottenburg-Wilmersdorf, 10623', 'Charlottenburg-Wilmersdorf, 10719', 'Charlottenburg-Wilmersdorf, 14050', 'Charlottenburg-Wilmersdorf, 10625', 'Charlottenburg-Wilmersdorf, 10777', 'Charlottenburg-Wilmersdorf, 14052', 'Charlottenburg-Wilmersdorf, 10627', 'Charlottenburg-Wilmersdorf, 10779', 'Charlottenburg-Wilmersdorf, 14053', 'Charlottenburg-Wilmersdorf, 10629', 'Charlottenburg-Wilmersdorf, 10787', 'Charlottenburg-Wilmersdorf, 14055', 'Charlottenburg-Wilmersdorf, 10707', 'Charlottenburg-Wilmersdorf, 10789', 'Charlottenburg-Wilmersdorf, 14057', 'Charlottenburg-Wilmersdorf, 10709', 'Charlottenburg-Wilmersdorf, 10825', 'Charlottenburg-Wilmersdorf, 14059', 'Spandau, 13581', 'Spandau, 13627', 'Spandau, 13583', 'Spandau, 13629', 'Spandau, 13585', 'Spandau, 14052', 'Spandau, 13587', 'Spandau, 14089', 'Spandau, 13589', 'Spandau, 13591', 'Spandau, 13593', 'Spandau, 13595', 'Spandau, 13597', 'Spandau, 13599', 'Steglitz-Zehlendorf, 12157', 'Steglitz-Zehlendorf, 12247', 'Steglitz-Zehlendorf, 14193', 'Steglitz-Zehlendorf, 12161', 'Steglitz-Zehlendorf, 12249', 'Steglitz-Zehlendorf, 14195', 'Steglitz-Zehlendorf, 12163', 'Steglitz-Zehlendorf, 12277', 'Steglitz-Zehlendorf, 14197', 'Steglitz-Zehlendorf, 12165', 'Steglitz-Zehlendorf, 12279', 'Steglitz-Zehlendorf, 14199', 'Steglitz-Zehlendorf, 12167', 'Steglitz-Zehlendorf, 14109', 'Steglitz-Zehlendorf, 12169', 'Steglitz-Zehlendorf, 14129', 'Steglitz-Zehlendorf, 12203', 'Steglitz-Zehlendorf, 14163', 'Steglitz-Zehlendorf, 12205', 'Steglitz-Zehlendorf, 14165', 'Steglitz-Zehlendorf, 12207', 'Steglitz-Zehlendorf, 14167', 'Steglitz-Zehlendorf, 12209', 'Steglitz-Zehlendorf, 14169', 'Tempelhof-Schöneberg, 10777', 'Tempelhof-Schöneberg, 10829', 'Tempelhof-Schöneberg, 12161', 'Tempelhof-Schöneberg, 14197', 'Tempelhof-Schöneberg, 10779', 'Tempelhof-Schöneberg, 10965', 'Tempelhof-Schöneberg, 12163', 'Tempelhof-Schöneberg, 10781', 'Tempelhof-Schöneberg, 12099', 'Tempelhof-Schöneberg, 12169', 'Tempelhof-Schöneberg, 10783', 'Tempelhof-Schöneberg, 12101', 'Tempelhof-Schöneberg, 12249', 'Tempelhof-Schöneberg, 10785', 'Tempelhof-Schöneberg, 12103', 'Tempelhof-Schöneberg, 12277', 'Tempelhof-Schöneberg, 10787', 'Tempelhof-Schöneberg, 12105', 'Tempelhof-Schöneberg, 12279', 'Tempelhof-Schöneberg, 10789', 'Tempelhof-Schöneberg, 12107', 'Tempelhof-Schöneberg, 12305', 'Tempelhof-Schöneberg, 10823', 'Tempelhof-Schöneberg, 12109', 'Tempelhof-Schöneberg, 12307', 'Tempelhof-Schöneberg, 10825', 'Tempelhof-Schöneberg, 12157', 'Tempelhof-Schöneberg, 12309', 'Tempelhof-Schöneberg, 10827', 'Tempelhof-Schöneberg, 12159', 'Tempelhof-Schöneberg, 12347', 'Neukölln, 10965', 'Neukölln, 12059', 'Neukölln, 12359', 'Neukölln, 10967', 'Neukölln, 12099', 'Neukölln, 12043', 'Neukölln, 12107', 'Neukölln, 12045', 'Neukölln, 12305', 'Neukölln, 12047', 'Neukölln, 12347', 'Neukölln, 12049', 'Neukölln, 12349', 'Neukölln, 12051', 'Neukölln, 12351', 'Neukölln, 12053', 'Neukölln, 12353', 'Neukölln, 12055', 'Neukölln, 12355', 'Neukölln, 12057', 'Neukölln, 12357', 'Treptow-Köpenick, 12435', 'Treptow-Köpenick, 12557', 'Treptow-Köpenick, 12437', 'Treptow-Köpenick, 12559', 'Treptow-Köpenick, 12439', 'Treptow-Köpenick, 12587', 'Treptow-Köpenick, 12459', 'Treptow-Köpenick, 12589', 'Treptow-Köpenick, 12487', 'Treptow-Köpenick, 12623', 'Treptow-Köpenick, 12489', 'Treptow-Köpenick, 12524', 'Treptow-Köpenick, 12526', 'Treptow-Köpenick, 12527', 'Treptow-Köpenick, 12555', 'Marzahn-Hellersdorf, 12555', 'Marzahn-Hellersdorf, 12687', 'Marzahn-Hellersdorf, 12619', 'Marzahn-Hellersdorf, 12689', 'Marzahn-Hellersdorf, 12621', 'Marzahn-Hellersdorf, 12623', 'Marzahn-Hellersdorf, 12627', 'Marzahn-Hellersdorf, 12629', 'Marzahn-Hellersdorf, 12679', 'Marzahn-Hellersdorf, 12681', 'Marzahn-Hellersdorf, 12683', 'Marzahn-Hellersdorf, 12685', 'Lichtenberg, 10315', 'Lichtenberg, 13057', 'Lichtenberg, 10317', 'Lichtenberg, 13059', 'Lichtenberg, 10318', 'Lichtenberg, 10319', 'Lichtenberg, 10365', 'Lichtenberg, 10367', 'Lichtenberg, 10369', 'Lichtenberg, 13051', 'Lichtenberg, 13053', 'Lichtenberg, 13055', 'Reinickendorf, 13403', 'Reinickendorf, 13503', 'Reinickendorf, 13405', 'Reinickendorf, 13505', 'Reinickendorf, 13407', 'Reinickendorf, 13507', 'Reinickendorf, 13409', 'Reinickendorf, 13509', 'Reinickendorf, 13435', 'Reinickendorf, 13599', 'Reinickendorf, 13437', 'Reinickendorf, 13629', 'Reinickendorf, 13439', 'Reinickendorf, 13465', 'Reinickendorf, 13467', 'Reinickendorf, 13469']

# Different search queries to get more results
search_queries = []

for type in type_of_restaurans:
    for district in districts:
        search_queries.append(f"{type} restaurants in {district} Berlin")

all_restaurants = []
seen_ids = set()

for query in search_queries:
    print(f"Searching: {query}")

    body = {
        "textQuery": query,
        "pageSize": 50
    }

    response = requests.post(BASE_URL, headers=headers, json=body)
    data = response.json()

    for place in data.get("places", []):
        id = place.get("id")

        # Skip duplicates
        if id in seen_ids:
            continue
        seen_ids.add(id)

        populartimes_data = populartimes.get_id(GCP_API_KEY, id)

        all_restaurants.append({
            "id": populartimes_data.get("id") or id,
            "name": populartimes_data.get("name") or place.get("displayName", {}).get("text"),
            "type_of_food": query.split(" ")[0],
            "address": populartimes_data.get("address") or place.get("formattedAddress"),
            "short_address": place.get("shortFormattedAddress"),
            "location": populartimes_data.get("coordinates") or place.get("location"),
            "district": query.split("in ")[1].split(",")[0],
            "lat": (populartimes_data.get("coordinates") or {}).get("lat") if populartimes_data.get("coordinates") else None,
            "lng": (populartimes_data.get("coordinates") or {}).get("lng") if populartimes_data.get("coordinates") else None,
            "national_phone": place.get("nationalPhoneNumber"),
            "international_phone": populartimes_data.get("international_phone_number") or place.get("internationalPhoneNumber"),
            "website": place.get("websiteUri"),
            "rating": populartimes_data.get("rating") or place.get("rating"),
            "total_ratings": populartimes_data.get("rating_n") or place.get("userRatingCount"),
            "price_level": place.get("priceLevel"),
            "business_status": place.get("businessStatus"),
            "types": populartimes_data.get("types") or place.get("types"),
            "current_opening_hours": place.get("currentOpeningHours"),
            "regular_opening_hours": place.get("regularOpeningHours"),
            "editorial_summary": place.get("editorialSummary"),
            "accessibility_options": place.get("accessibilityOptions"),
            "parking_options": place.get("parkingOptions"),
            "payment_options": place.get("paymentOptions"),
            "takeout": place.get("takeout"),
            "delivery": place.get("delivery"),
            "dine_in": place.get("dineIn"),
            "google_maps_uri": place.get("googleMapsUri"),
            "current_popularity": populartimes_data.get("current_popularity"),
            "populartimes": populartimes_data.get("populartimes"),
            "time_wait": populartimes_data.get("time_wait")
        })

    print(f"Total unique restaurants: {len(all_restaurants)}")

    time.sleep(1)

df = pd.DataFrame(all_restaurants)
print(f"\nFinal count: {len(df)} restaurants")
print(df.head(10))

df.to_csv("restaurants.csv", index=False, encoding="utf-8-sig")
print("\nSaved to restaurants.csv")

Searching: Korean restaurants in Mitte, 10115 Berlin
Total unique restaurants: 20
Searching: Korean restaurants in Mitte, 10559 Berlin
Total unique restaurants: 26
Searching: Korean restaurants in Mitte, 13355 Berlin
Total unique restaurants: 27
Searching: Korean restaurants in Mitte, 10117 Berlin
Total unique restaurants: 31
Searching: Korean restaurants in Mitte, 10623 Berlin
Total unique restaurants: 44
Searching: Korean restaurants in Mitte, 13357 Berlin
Total unique restaurants: 46
Searching: Korean restaurants in Mitte, 10119 Berlin
Total unique restaurants: 46
Searching: Korean restaurants in Mitte, 10785 Berlin
Total unique restaurants: 47
Searching: Korean restaurants in Mitte, 13359 Berlin
Total unique restaurants: 49
Searching: Korean restaurants in Mitte, 10178 Berlin
Total unique restaurants: 49
Searching: Korean restaurants in Mitte, 10787 Berlin
Total unique restaurants: 51
Searching: Korean restaurants in Mitte, 13405 Berlin
Total unique restaurants: 52
Searching: Korea

In [15]:
import requests
import pandas as pd
import time
import populartimes

BASE_URL = "https://places.googleapis.com/v1/places:searchText"

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": GCP_API_KEY,
    "X-Goog-FieldMask": (
        "places.id,"
        "places.displayName,"
        "nextPageToken"
    )
}

company_types = [
    "mbH", "GmbH", "AG", "AG &Co", "Fa.", "gGmbH", "UG", "GbR", "OHG", "KG", "GmbH & Co. KG", "KGaA"
]

districts = ['Mitte, 10115', 'Mitte, 10559', 'Mitte, 13355', 'Mitte, 10117', 'Mitte, 10623', 'Mitte, 13357', 'Mitte, 10119', 'Mitte, 10785', 'Mitte, 13359', 'Mitte, 10178', 'Mitte, 10787', 'Mitte, 13405', 'Mitte, 10179', 'Mitte, 10963', 'Mitte, 13407', 'Mitte, 10435', 'Mitte, 10969', 'Mitte, 13409', 'Mitte, 10551', 'Mitte, 13347', 'Mitte, 10553', 'Mitte, 13349', 'Mitte, 10555', 'Mitte, 13351', 'Mitte, 10557', 'Mitte, 13353', 'Friedrichshain-Kreuzberg, 10179', 'Friedrichshain-Kreuzberg, 10967', 'Friedrichshain-Kreuzberg, 10243', 'Friedrichshain-Kreuzberg, 10969', 'Friedrichshain-Kreuzberg, 10245', 'Friedrichshain-Kreuzberg, 10997', 'Friedrichshain-Kreuzberg, 10247', 'Friedrichshain-Kreuzberg, 10999', 'Friedrichshain-Kreuzberg, 10249', 'Friedrichshain-Kreuzberg, 12045', 'Friedrichshain-Kreuzberg, 10367', 'Friedrichshain-Kreuzberg, 10785', 'Friedrichshain-Kreuzberg, 10961', 'Friedrichshain-Kreuzberg, 10963', 'Friedrichshain-Kreuzberg, 10965', 'Pankow, 10119', 'Pankow, 13053', 'Pankow, 13187', 'Pankow, 10247', 'Pankow, 13086', 'Pankow, 13189', 'Pankow, 10249', 'Pankow, 13088', 'Pankow, 10405', 'Pankow, 13089', 'Pankow, 10407', 'Pankow, 13125', 'Pankow, 10409', 'Pankow, 13127', 'Pankow, 10435', 'Pankow, 13129', 'Pankow, 10437', 'Pankow, 13156', 'Pankow, 10439', 'Pankow, 13158', 'Pankow, 13051', 'Pankow, 13159', 'Charlottenburg-Wilmersdorf, 10553', 'Charlottenburg-Wilmersdorf, 10711', 'Charlottenburg-Wilmersdorf, 13353', 'Charlottenburg-Wilmersdorf, 14193', 'Charlottenburg-Wilmersdorf, 10585', 'Charlottenburg-Wilmersdorf, 10713', 'Charlottenburg-Wilmersdorf, 13597', 'Charlottenburg-Wilmersdorf, 14195', 'Charlottenburg-Wilmersdorf, 10587', 'Charlottenburg-Wilmersdorf, 10715', 'Charlottenburg-Wilmersdorf, 13627', 'Charlottenburg-Wilmersdorf, 14197', 'Charlottenburg-Wilmersdorf, 10589', 'Charlottenburg-Wilmersdorf, 10717', 'Charlottenburg-Wilmersdorf, 13629', 'Charlottenburg-Wilmersdorf, 14199', 'Charlottenburg-Wilmersdorf, 10623', 'Charlottenburg-Wilmersdorf, 10719', 'Charlottenburg-Wilmersdorf, 14050', 'Charlottenburg-Wilmersdorf, 10625', 'Charlottenburg-Wilmersdorf, 10777', 'Charlottenburg-Wilmersdorf, 14052', 'Charlottenburg-Wilmersdorf, 10627', 'Charlottenburg-Wilmersdorf, 10779', 'Charlottenburg-Wilmersdorf, 14053', 'Charlottenburg-Wilmersdorf, 10629', 'Charlottenburg-Wilmersdorf, 10787', 'Charlottenburg-Wilmersdorf, 14055', 'Charlottenburg-Wilmersdorf, 10707', 'Charlottenburg-Wilmersdorf, 10789', 'Charlottenburg-Wilmersdorf, 14057', 'Charlottenburg-Wilmersdorf, 10709', 'Charlottenburg-Wilmersdorf, 10825', 'Charlottenburg-Wilmersdorf, 14059', 'Spandau, 13581', 'Spandau, 13627', 'Spandau, 13583', 'Spandau, 13629', 'Spandau, 13585', 'Spandau, 14052', 'Spandau, 13587', 'Spandau, 14089', 'Spandau, 13589', 'Spandau, 13591', 'Spandau, 13593', 'Spandau, 13595', 'Spandau, 13597', 'Spandau, 13599', 'Steglitz-Zehlendorf, 12157', 'Steglitz-Zehlendorf, 12247', 'Steglitz-Zehlendorf, 14193', 'Steglitz-Zehlendorf, 12161', 'Steglitz-Zehlendorf, 12249', 'Steglitz-Zehlendorf, 14195', 'Steglitz-Zehlendorf, 12163', 'Steglitz-Zehlendorf, 12277', 'Steglitz-Zehlendorf, 14197', 'Steglitz-Zehlendorf, 12165', 'Steglitz-Zehlendorf, 12279', 'Steglitz-Zehlendorf, 14199', 'Steglitz-Zehlendorf, 12167', 'Steglitz-Zehlendorf, 14109', 'Steglitz-Zehlendorf, 12169', 'Steglitz-Zehlendorf, 14129', 'Steglitz-Zehlendorf, 12203', 'Steglitz-Zehlendorf, 14163', 'Steglitz-Zehlendorf, 12205', 'Steglitz-Zehlendorf, 14165', 'Steglitz-Zehlendorf, 12207', 'Steglitz-Zehlendorf, 14167', 'Steglitz-Zehlendorf, 12209', 'Steglitz-Zehlendorf, 14169', 'Tempelhof-Schöneberg, 10777', 'Tempelhof-Schöneberg, 10829', 'Tempelhof-Schöneberg, 12161', 'Tempelhof-Schöneberg, 14197', 'Tempelhof-Schöneberg, 10779', 'Tempelhof-Schöneberg, 10965', 'Tempelhof-Schöneberg, 12163', 'Tempelhof-Schöneberg, 10781', 'Tempelhof-Schöneberg, 12099', 'Tempelhof-Schöneberg, 12169', 'Tempelhof-Schöneberg, 10783', 'Tempelhof-Schöneberg, 12101', 'Tempelhof-Schöneberg, 12249', 'Tempelhof-Schöneberg, 10785', 'Tempelhof-Schöneberg, 12103', 'Tempelhof-Schöneberg, 12277', 'Tempelhof-Schöneberg, 10787', 'Tempelhof-Schöneberg, 12105', 'Tempelhof-Schöneberg, 12279', 'Tempelhof-Schöneberg, 10789', 'Tempelhof-Schöneberg, 12107', 'Tempelhof-Schöneberg, 12305', 'Tempelhof-Schöneberg, 10823', 'Tempelhof-Schöneberg, 12109', 'Tempelhof-Schöneberg, 12307', 'Tempelhof-Schöneberg, 10825', 'Tempelhof-Schöneberg, 12157', 'Tempelhof-Schöneberg, 12309', 'Tempelhof-Schöneberg, 10827', 'Tempelhof-Schöneberg, 12159', 'Tempelhof-Schöneberg, 12347', 'Neukölln, 10965', 'Neukölln, 12059', 'Neukölln, 12359', 'Neukölln, 10967', 'Neukölln, 12099', 'Neukölln, 12043', 'Neukölln, 12107', 'Neukölln, 12045', 'Neukölln, 12305', 'Neukölln, 12047', 'Neukölln, 12347', 'Neukölln, 12049', 'Neukölln, 12349', 'Neukölln, 12051', 'Neukölln, 12351', 'Neukölln, 12053', 'Neukölln, 12353', 'Neukölln, 12055', 'Neukölln, 12355', 'Neukölln, 12057', 'Neukölln, 12357', 'Treptow-Köpenick, 12435', 'Treptow-Köpenick, 12557', 'Treptow-Köpenick, 12437', 'Treptow-Köpenick, 12559', 'Treptow-Köpenick, 12439', 'Treptow-Köpenick, 12587', 'Treptow-Köpenick, 12459', 'Treptow-Köpenick, 12589', 'Treptow-Köpenick, 12487', 'Treptow-Köpenick, 12623', 'Treptow-Köpenick, 12489', 'Treptow-Köpenick, 12524', 'Treptow-Köpenick, 12526', 'Treptow-Köpenick, 12527', 'Treptow-Köpenick, 12555', 'Marzahn-Hellersdorf, 12555', 'Marzahn-Hellersdorf, 12687', 'Marzahn-Hellersdorf, 12619', 'Marzahn-Hellersdorf, 12689', 'Marzahn-Hellersdorf, 12621', 'Marzahn-Hellersdorf, 12623', 'Marzahn-Hellersdorf, 12627', 'Marzahn-Hellersdorf, 12629', 'Marzahn-Hellersdorf, 12679', 'Marzahn-Hellersdorf, 12681', 'Marzahn-Hellersdorf, 12683', 'Marzahn-Hellersdorf, 12685', 'Lichtenberg, 10315', 'Lichtenberg, 13057', 'Lichtenberg, 10317', 'Lichtenberg, 13059', 'Lichtenberg, 10318', 'Lichtenberg, 10319', 'Lichtenberg, 10365', 'Lichtenberg, 10367', 'Lichtenberg, 10369', 'Lichtenberg, 13051', 'Lichtenberg, 13053', 'Lichtenberg, 13055', 'Reinickendorf, 13403', 'Reinickendorf, 13503', 'Reinickendorf, 13405', 'Reinickendorf, 13505', 'Reinickendorf, 13407', 'Reinickendorf, 13507', 'Reinickendorf, 13409', 'Reinickendorf, 13509', 'Reinickendorf, 13435', 'Reinickendorf, 13599', 'Reinickendorf, 13437', 'Reinickendorf, 13629', 'Reinickendorf, 13439', 'Reinickendorf, 13465', 'Reinickendorf, 13467', 'Reinickendorf, 13469']

# Different search queries to get more results
search_queries = []

for type in company_types:
    for district in districts:
        search_queries.append(f"Companies {type} in {district} Berlin")

all_companies = []
seen_ids = set()

for query in search_queries:
    print(f"Searching: {query}")

    body = {
        "textQuery": query,
        "pageSize": 50
    }

    response = requests.post(BASE_URL, headers=headers, json=body)
    data = response.json()

    for place in data.get("places", []):
        id = place.get("id")

        # Skip duplicates
        if id in seen_ids:
            continue
        seen_ids.add(id)

        all_companies.append({
            "id": id,
            "name": place.get("displayName", {}).get("text"),
            "district": query.split("in ")[1].split(",")[0],
        })

    print(f"Total unique companies: {len(all_companies)}")

df = pd.DataFrame(all_companies)
print(f"\nFinal count: {len(df)} companies")
print(df.head(10))

df.to_csv("companies.csv", index=False, encoding="utf-8-sig")
print("\nSaved to companies.csv")

Streaming output truncated to the last 5000 lines.
Total unique companies: 3127
Searching: Companies GmbH in Neukölln, 12099 Berlin
Total unique companies: 3127
Searching: Companies GmbH in Neukölln, 12043 Berlin
Total unique companies: 3128
Searching: Companies GmbH in Neukölln, 12107 Berlin
Total unique companies: 3128
Searching: Companies GmbH in Neukölln, 12045 Berlin
Total unique companies: 3129
Searching: Companies GmbH in Neukölln, 12305 Berlin
Total unique companies: 3129
Searching: Companies GmbH in Neukölln, 12047 Berlin
Total unique companies: 3129
Searching: Companies GmbH in Neukölln, 12347 Berlin
Total unique companies: 3131
Searching: Companies GmbH in Neukölln, 12049 Berlin
Total unique companies: 3131
Searching: Companies GmbH in Neukölln, 12349 Berlin
Total unique companies: 3131
Searching: Companies GmbH in Neukölln, 12051 Berlin
Total unique companies: 3131
Searching: Companies GmbH in Neukölln, 12351 Berlin
Total unique companies: 3131
Searching: Companies GmbH in 

In [ ]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"start_date": "2025-07-01",
	"end_date": "2025-10-01",
	"daily": ["weather_code", "temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_mean", "precipitation_sum", "snowfall_sum", "sunshine_duration"],
	"timezone": "auto",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(4).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(5).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(6).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(7).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["sunshine_duration"] = daily_sunshine_duration

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe.to_csv("weather.csv", index=False)
print("\nSaved to weather.csv")

In [ ]:
import requests

url = "https://de.wikipedia.org/wiki/Berlin"

# Set a browser user-agent to avoid 403 errors
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/131.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
response.raise_for_status()

# Now parse tables from the downloaded HTML
tables = pd.read_html(response.text)

print(f"Found {len(tables)} tables.")
df = tables[15] #15 contains what we want

df.columns = ["berlin_district", "population", "area_in_km", "pop_density"]

coords = {
    "Mitte": (52.5200, 13.4050),
    "Friedrichshain-Kreuzberg": (52.5090, 13.4310),
    "Pankow": (52.5667, 13.4167),
    "Charlottenburg-Wilmersdorf": (52.5078, 13.3039),
    "Spandau": (52.5511, 13.1997),
    "Steglitz-Zehlendorf": (52.4415, 13.2610),
    "Tempelhof-Schöneberg": (52.4669, 13.3500),
    "Neukölln": (52.4800, 13.4350),
    "Treptow-Köpenick": (52.4400, 13.5800),
    "Marzahn-Hellersdorf": (52.5400, 13.5750),
    "Lichtenberg": (52.5150, 13.4990),
    "Reinickendorf": (52.5880, 13.2870)
}

df["lat"] = df["berlin_district"].map(lambda x: coords.get(x, (None, None))[0])
df["lng"] = df["berlin_district"].map(lambda x: coords.get(x, (None, None))[1])

df=df.drop(index=12)
df.to_csv('berlin_district_info.csv', index=False, encoding="utf-8-sig")
print("\nSaved to berlin_district_info.csv")

In [ ]:
# Christmas market

from io import StringIO
import requests
import pandas as pd

url = "https://www.berlin.de/sen/web/service/maerkte-feste/weihnachtsmaerkte/index.php/index/all.csv?q="

print("Downloading CSV...")
response = requests.get(url)
response.raise_for_status()  # Check if download was successful

# Read CSV with semicolon separator
df = pd.read_csv(StringIO(response.text), sep=';')

print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

# Show basic info about the data
print(f"\nData types:")
print(df.dtypes)

In [ ]:
# Street food and folk festivals

from io import StringIO
import requests
import pandas as pd

url = "https://www.berlin.de/sen/web/service/maerkte-feste/strassen-volksfeste/index.php/index/all.csv?q="

print("Downloading CSV...")
response = requests.get(url)
response.raise_for_status()  # Check if download was successful

# Read CSV with semicolon separator
df = pd.read_csv(StringIO(response.text), sep=';')

print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn names:\n{df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())

# Show basic info about the data
print(f"\nData types:")
print(df.dtypes)